In [1]:
import warnings
warnings.filterwarnings("ignore")
import keras.utils as image
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
from tensorflow import keras
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers,models, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard
from tensorflow.keras.applications import ResNet50, MobileNetV3Small
from tensorflow.keras.applications import MobileNetV2, InceptionV3, VGG16 , VGG19
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization 
from tensorflow.keras.layers import Lambda
from pathlib import Path
import glob
import os
import cv2

tf.get_logger().setLevel('INFO')


In [9]:
import sklearn
print(sklearn.__version__)

1.5.2


In [4]:
print(tf.keras.__version__)

2.9.0


In [ ]:
if not tf.test.gpu_device_name():
    print('No GPU found')
else:
    print('Default GPU device: {}' .format(tf.test.gpu_device_name()))

## Loading Images Function

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)


    return images

## Label Images

In [ ]:
def data(folders):
    imgs = []
    labels = []
    names = []
    for folder in folders:
        for img in load_images_from_folder(folder):
            imgs.append(img)

            if 'CaS' in folder:
                labels.append('CaS')
            elif 'CoS' in folder:
                labels.append('CoS')
            elif 'Gum' in folder:
                labels.append('Gum')
            elif 'MC' in folder:
                labels.append('MC')
            elif 'OC' in folder:
                labels.append('OC')
            elif 'OLP' in folder:
                labels.append('OLP')
            elif 'OT' in folder:
                labels.append('OT')
            else:
                print("error")

        for name in os.listdir(folder):
            names.append(os.path.join(folder, name))

    names = pd.Series(names, name= 'name')
    imgs = pd.Series(imgs, name='image', dtype= np.float32)
    labels = pd.Series(labels, name="label")

    image_df = pd.concat([names, imgs, labels], axis=1)

    return image_df


## Loading Images

In [ ]:
path = "Teeth_Dataset/Training"
folders = [os.path.join(path, x) for x in os.listdir(path)]
train_df = data(folders)

In [ ]:
path = "Teeth_Dataset/Testing"
folders = [os.path.join(path, x) for x in os.listdir(path)]
test_df = data(folders)

In [ ]:
path = "Teeth_Dataset/Validation"
folders = [os.path.join(path, x) for x in os.listdir(path)]
validation_df = data(folders)

# Visulaization

In [ ]:
label_counts = train_df["label"].value_counts()

In [ ]:
plt.figure(figsize=(15, 6))
sns.barplot(x=label_counts.index, y=label_counts.values, alpha=0.8, palette='dark:salmon_r')

plt.title('Distribution of Labels in Image Dataset', fontsize=16)
plt.xlabel('Label', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.show()

In [ ]:
plt.figure(figsize= (20, 10))
random_values = np.random.randint(0, len(train_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize = (10, 10))

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_df.name[random_values[i]]))
    ax.set_title(train_df.label[random_values[i]])
    plt.tight_layout()
plt.show()

## Augmentation

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

In [ ]:
train_path = train_df.pop("name").astype(str)
valid_path = validation_df.pop("name").astype(str)
test_path = test_df.pop("name").astype(str)



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(
    "Teeth_Dataset/Training",
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = train_datagen.flow_from_directory(
    "Teeth_Dataset/Validation",
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_datagen_generator = train_datagen.flow_from_directory(
    "Teeth_Dataset/Testing",
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

## From Scratch

In [ ]:
# input_shape = [224, 224, 3]

# model = tf.keras.Sequential([

#     layers.Conv2D(filters= 64, kernel_size= (3,3), activation= "relu", input_shape= input_shape ),
#     layers.MaxPool2D(pool_size=(2,2), strides= 2),
#     layers.Conv2D(filters= 128, kernel_size= (3,3), activation= "relu", input_shape= input_shape ),
#     layers.MaxPool2D(pool_size=(2,2), strides= 2),
#     layers.Conv2D(filters= 256, kernel_size= (3,3), activation= "relu", input_shape= input_shape ),
#     layers.MaxPool2D(pool_size=(2,2), strides= 2),

#     layers.Flatten(),
#     layers.Dense(units= 1000, activation= "relu"),
#     layers.Dropout(.5),
#     layers.Dense(units= 200, activation= "relu"),


#     layers.Dense(7, activation= "softmax")


# ])

# model.summary()

# model.compile(
#     optimizer= keras.optimizers.Adamax(learning_rate=.0001),
#     loss = "categorical_crossentropy",
#     metrics= ["accuracy"]
# )

In [ ]:
# # Create checkpoint callback
# checkpoint_path = "model_checkpoint.keras"
# checkpoint_callback = ModelCheckpoint(checkpoint_path,
#                                       monitor="val_accuracy",
#                                       save_best_only=True)

# # Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
# early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
#                                patience = 10,
#                                restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [ ]:
# history = model.fit(
#     train_generator,
#     epochs=10,
#     validation_data= valid_generator,
#     callbacks=[
#         early_stopping,
#         reduce_lr, 
#         checkpoint_callback
#     ]
#  )

# Transfer Learning

In [ ]:
input_shape = [224, 224, 3]
vgg = VGG16(input_shape= input_shape, include_top= False, classes=1000, pooling= "max")
vgg.summary()

In [ ]:
# Removing Dense Layers
model_without_last_layer = tf.keras.Model(inputs=vgg.input, outputs=vgg.layers[-2].output)
model_without_last_layer.summary()

In [ ]:
# Freezing
for lay in model_without_last_layer.layers:
    lay.trainable = False

In [ ]:
# Create checkpoint callback
checkpoint_path = "VGG16_Checkpoint.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-10)

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x, input_shape=(224, 224, 3)))

model.add(model_without_last_layer)

model.add(BatchNormalization())
model.add(Conv2D(350, (3, 3), activation= "relu", padding= "same"))
model.add(tf.keras.layers.SpatialDropout2D(.2))

# model.add(BatchNormalization())
model.add(Conv2D(100, (3, 3), activation= "relu"))
model.add(tf.keras.layers.SpatialDropout2D(.2))


model.add(Conv2D(50, (3, 3), activation= "relu"))
model.add(tf.keras.layers.SpatialDropout2D(.2))
model.add(Flatten())


model.add(Dense(450, activation= "relu"))
model.add(layers.Dropout(.2))

model.add(Dense(150, activation= "relu"))
model.add(layers.Dropout(.2))


model.add(Dense(50, activation= "relu"))
model.add(layers.Dropout(.2))

model.add(Dense(7, activation= "softmax"))

model.summary()

## Training

In [ ]:
model.compile(
    optimizer= keras.optimizers.Adam(learning_rate=0.01),
    loss = "categorical_crossentropy",
    metrics= ["accuracy"]
)


history = model.fit(
    train_generator,
    epochs=10,
    validation_data= valid_generator,
    callbacks=[
        early_stopping,
        reduce_lr, 
        checkpoint_callback
    ]
 )

## Evaluation

In [ ]:
results = model.evaluate(test_datagen_generator , verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.title('Training and validation loss')
plt.legend()
plt.show()

## Saving Model

In [ ]:
# keras.models.save_model(model, "vgg16_model.h5")

## Prediction

In [ ]:
model = keras.models.load_model("vgg16_model.h5")

In [ ]:
def model_predict(img_path, model):

    # Preprocessing the image
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.image.resize(img, [224,224])
    img = img/255.0
    #img = preprocess_input(x)

    y = model.predict(img)

    return np.argmax(y)

In [ ]:
model_predict(test_path[1027], model)